Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

# Week 9 - Clustering

For this assignment, you will use the *digits* dataset.

The components of this assignment are:
1. Load the digits dataset into a Pandas data frame.
1. Preprocess the digits dataset.
1. Examine how many clusters you may need using the elbow method.
1. Cluster the digits dataset using K-means and evaluate using adjusted rand index.
1. Optional: See how the ARI changes as you alter the K-means parameters.
1. Optional: Use clustering as a classifier and compare to Random Forest.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## Task 1. Load the digits dataset into a Pandas data frame.

The digits dataset represents images of handwritten digits. Each image is a 32x32 bitmap. To create a numerical dataset representing the images, each image was divided into non-overlapping blocks of 4x4, and the number of set pixels in each block is counted.

Thus, each image is represented by an 8x8 matrix of integers in the range 0-16. Each sample in the dataset therefore has 64 attributes. There are a total of 1,797 samples in the dataset we will use.

Therefore, the digits dataset should be loaded into a Pandas dataframe with 1,797 rows and 64 columns.

In this task, you will load the digits dataset (http://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tes) into a Pandas dataframe. Note that the final column in this dataset is the correct label (an integer in the range 0-9).

Load the attributes into a Pandas dataframe named **digits**. This dataframe should *not* include the final column, as we only want to load the features here.

Instead, save the final column into a Pandas series named **labels**.

In [66]:
import pandas as pd
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/optdigits/optdigits.tes"
df = pd.read_csv(url, header=-1)
digits = df.iloc[:, :-1]
labels = df.iloc[:,-1]

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0,0,5,13,9,1,0,0,0,0,...,0,0,0,0,6,13,10,0,0,0
1,0,0,0,12,13,5,0,0,0,0,...,0,0,0,0,0,11,16,10,0,0
2,0,0,0,4,15,12,0,0,0,0,...,5,0,0,0,0,3,11,16,9,0
3,0,0,7,15,13,1,0,0,0,8,...,9,0,0,0,7,13,13,9,0,0
4,0,0,0,1,11,0,0,0,0,0,...,0,0,0,0,0,2,16,4,0,0


In [54]:
assert digits.shape == (1797, 64)
assert labels.shape == (1797,)
assert digits.sum().sum() == 561718
assert all((0 <= labels) & (labels <= 9))

## Task 2. Preprocess the digits dataset.

Standardize the data so that each column has a mean of (or very close to) zero and a standard deviation of (or very close to) 1. Make sure the resulting data is a Pandas dataframe (if it is not, convert it to one using pd.DataFrame).

Note that some of the columns are all zeros across all samples. In this case, you can leave the column as is. In theory you could remove those columns as they do not contribute any information to clustering, but in our case they do not affect the results so we will keep them. For this reason, our assertions makes sure that all the columns have a standard variation of one **except** those columns that are all zeros.

In [97]:
import numpy as np
digits = ((digits - digits.mean()) / digits.std()).fillna(0)
digits.replace([np.inf, -np.inf], np.nan)
digits = digits.fillna(0)

In [98]:
assert digits.shape == (1797, 64)
assert all(np.isclose(digits.mean(), 0))
assert all(np.isclose(digits.std(ddof=0), 1) == (digits.sum() != 0))
assert np.isclose(digits.sum().sum(), 0)

AssertionError: 

## Task 3. Examine how many clusters you may need using the elbow method.

The K-means algorithm requires us to specify the number of clusters. One way to assess this is using the elbow method.

Use the elbow method and see if there is a clear number of clusters that make sense in the digits dataset. Note that this is just for exploratory purposes; we know the number of clusters should be 10, and that is what we will use in the future components of this exercise.

The elbow method consists of visualizing the sum of squared distances between samples and their closest cluster center. This is done by creating a scatter plot where the number of clusters used in K-means is provided on the X axis and the sum of squared distances is provided on the Y axis. If there is a clear inflection point, that can be an indicator of the ideal number of clusters.

Run the K-means algorithm on the digits dataset 25 times. In the first run set the number of clusters to 1, in the second to 2, and so on through 25. For each run, collect the sum of squared distances. Save these into a list named **ssds**.

Create an elbow plot and examine whether you can see a clear inflection point.

**Note:**
In order for our assertions to work, make sure you use the default parameters for KMeans, with the exception of random_state, which you should set to 126, i.e. to run with one cluster:
```python
KMeans(n_clusters=1, random_state=126)
```

and so forth.

In [99]:
from sklearn.cluster import KMeans

def plot_elbow(dataset, max_clusters):
    """Plot elbow curve for k-means."""
    inertias = []
    for i in range(1, max_clusters + 1):
        kmeans = KMeans(n_clusters=i, random_state=768797)
        kmeans.fit(dataset)
        inertias.append(kmeans.inertia_)

    plt.plot(range(1, max_clusters + 1), inertias)
    plt.title("Elbow Plot")
    plt.xlabel("K")
    plt.ylabel("SSD")

for i in range(25):
    model = KMeans(
        n_clusters=i,
        init='k-means++', # default, also: random
        n_init=10, # default, also: 5, 1
        max_iter=300, # default
        tol=1e-4, # default
        random_state=123124,
    )
    model.fit(digits)
    cluster_labels = model.predict(digits)
    plot_elbow(digits, i)

/Users/syedather/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:88: RuntimeWarning: divide by zero encountered in log
  n_local_trials = 2 + int(np.log(n_clusters))


OverflowError: cannot convert float infinity to integer

In [ ]:
assert len(ssds) == 25
assert np.isclose(np.mean(ssds), 68158.8514102)

## Task 4. Cluster the digits dataset using K-means and evaluate using adjusted rand index.

Go ahead and cluster the dataset using K-means. 

Set the number of clusters to ten and and the random_state to 126:
```python
KMeans(n_clusters=10, random_state=126)
```

Evaluate your clusters using the ARI. You can import the ARI function from sklearn.metrics:

```python
from sklearn.metrics import adjusted_rand_score
```

Save the ARI to the variable *score*. Save the KMeans model to the variable *model*.

In [ ]:
# YOUR CODE HERE
pass

In [ ]:
assert isinstance(score, float)
assert np.isclose(score, 0.477717062354)
assert len(model.labels_) == 1797
assert sum(model.labels_) == 5881

## Task 5. How does the ARI change when you alter the K-means parameters?
### Optional and ungraded.

Explore the effect on the ARI of changing the KMeans parameters, such as: init, n_init, max_iter.

Compare to other evaluations, such as: homogeneity score, completeness score, and the silhouette coefficient.

## Task 6. Clustering as a classifier?
### Optional and ungraded.

Can you use clustering as a classifier for the digits dataset? How does it compare to some of the classifiers you used last week, such as random forest?